In [1]:
import os
import sys

# adding classes folder to system path
sys.path.insert(0, os.path.abspath('..') + '/gispy')

import pandas as pd
from scipy.stats import zscore
from gist import GIS

In [2]:
df = pd.read_csv("../data/mturk_v2.csv")

In [3]:
a = GIS().score(df, wolfe=True)

In [4]:
a.head(5)

,TextID,DESPC,DESSC,DESWC,DESPL,DESPLd,DESSL,DESSLd,DESWLsy,DESWLsyd,...,WRDHYPv,WRDHYPnv,RDFRE,RDFKGL,RDL2,zSMCAUSlsa,zSMCAUSwn,zWRDIMGc,zWRDHYPnv,GIS
0,C:\gwu\cohmetrix\CohMetrix2021\coh_input\302U8...,5,10,209,2.000,1.000,21.200001,8.638,1.651,0.969,...,2.033,2.405,45.946999,12.043,6.208,-1.125,-1.479167,0.656158,2.161538,-5.498529
1,C:\gwu\cohmetrix\CohMetrix2021\coh_input\306W7...,7,12,311,1.714,0.756,26.167000,8.145,1.486,0.897,...,1.671,2.175,54.813999,12.052,6.363,-0.950,-0.031250,1.735856,1.276923,-5.244529
2,C:\gwu\cohmetrix\CohMetrix2021\coh_input\307L9...,7,10,291,1.429,0.535,29.299999,13.166,1.722,1.090,...,1.671,2.028,31.617001,16.079,8.662,-0.100,-1.000000,0.307193,0.711539,-0.199732
3,C:\gwu\cohmetrix\CohMetrix2021\coh_input\307L9...,3,7,196,2.333,0.577,28.285999,8.321,1.449,0.818,...,1.561,1.628,55.830002,12.428,24.882,-0.450,1.864583,-0.391654,-0.826923,-1.583006
4,C:\gwu\cohmetrix\CohMetrix2021\coh_input\308KJ...,10,13,223,1.300,0.483,17.153999,6.230,1.251,0.545,...,1.388,1.459,83.588997,5.862,20.775,-1.250,1.020834,0.278787,-1.476923,-1.139697


In [5]:
a.to_csv('mturk_v2.csv')